<a href="https://colab.research.google.com/github/jhyoo78/jhyoo78/blob/main/synonym_antonym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk  # natural Language toolkit(python)
nltk.download('wordnet')
nltk.download('omw-1.4')    #  Open Multilingual Wordnet collection 1.4
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
#  참고 -->  https://www.holisticseo.digital/python-seo/nltk/wordnet
def synonym_antonym_extractor(phrase):
     from nltk.corpus import wordnet
     synonyms = []
     antonyms = []

     for syn in wordnet.synsets(phrase):
          for l in syn.lemmas():
               synonyms.append(l.name())
               if l.antonyms():
                    antonyms.append(l.antonyms()[0].name())

     print(set(synonyms))
     print(set(antonyms))


In [ ]:
synonym_antonym_extractor(phrase="up")

In [ ]:
# 유사도 계산하기 ---> https://www.nltk.org/howto/wordnet.html
from nltk.corpus import wordnet
dog = wordnet.synset('dog.n.01')
cat = wordnet.synset('cat.n.01')
up = wordnet.synset('up.r.01')    # r= 부사(adverb), a= 형용사(adjective), n=noun, v=verb, s=adjective satellite (antonym이 없는 단어를 의미한다. 특이한 경우인데, wn에서 adjective의 경우는 (adjective, verb, antonym)의 triple의 형태로 관리되는데, anotynym이 없을 경우만 따로 adjective satellite 로 관리한다.)
down = wordnet.synset('down.r.01')

dog.path_similarity(cat)
dog.lch_similarity(cat)
#print(wordnet.path_similarity(dog, cat))
#print(wordnet.lch_similarity(up, down))   # Leacock-Chodorow Similarity, 이 관계는 -log(p/2d)으로 주어짐. 단, p는 shortest path length 이고 d 는 the taxonomy depth 이다.

a1= wordnet.synsets('starting')  # 단어의 원형이 출력됨. 예: 'starting'을 입력하면 'start'가 출력됨
a2= wordnet.synsets('started')
print("a1=", a1)
print("a2=", a2)
print(a1[0])
b1= wordnet.synset(a1[0].name())
b2= wordnet.synset(a2[0].name())
print("a1[0], a2[0] =", a1[0], a2[0])

# print(wordnet.lch_similarity(b1, b2) )  # 두 단어가 같은 POS 를 가져야 비교 가능함. 즉, 명사와 동사는 유사도 계산 불가함.

a1= wordnet.synsets('up', pos='n')        # POS에 해당하는 lemma만 출력
a2= wordnet.synsets('down', pos='n')
print("a1, a2=", a1, a2)                  # up의 동의어는 없음 --> []으로 출력됨.


In [ ]:
# 함수 없이 동의어 찾기
from nltk.corpus import wordnet
synonyms = []

print(wordnet.synsets("love"))

for syn in wordnet.synsets("love"):
    for i in syn.lemmas():
        synonyms.append(i.name())

print(set(synonyms))


In [ ]:
# 반의어 찾기

from nltk.corpus import wordnet
antonyms = []

for syn in wordnet.synsets("failed"):
    for i in syn.lemmas():
         if i.antonyms():
              antonyms.append(i.antonyms()[0].name())

print(set(antonyms))


In [ ]:
# https://www.holisticseo.digital/python-seo/nltk/wordnet
# 문장 내의 단에 대한 동의어 반의어 찾기

def text_parser_synonym_antonym_finder(text:str):
     from nltk.tokenize import word_tokenize
     from nltk.corpus import wordnet
     from collections import defaultdict
     import pprint

     tokens = word_tokenize(text)   # nltk의 tokenizer
     synonyms = defaultdict(list)
     antonyms = defaultdict(list)
     for token in tokens:
          for syn in wordnet.synsets(token):
               for i in syn.lemmas():
                    #synonyms.append(i.name())
                    #print(f'{token} synonyms are: {i.name()}')
                    synonyms[token].append(i.name())
                    if i.antonyms():
                         #antonyms.append(i.antonyms()[0].name())
                         #print(f'{token} antonyms are: {i.antonyms()[0].name()}')
                         antonyms[token].append(i.antonyms()[0].name())
     #pprint.pprint(dict(synonyms))   #################### 동의어 출력이 너무 많아서...............반의어만 보기 위해서...
     pprint.pprint(dict(antonyms))

     '''
     synonym_output = pprint.pformat((dict(synonyms)))
     antonyms_output = pprint.pformat((dict(antonyms)))
     with open(str(text[:5]) + ".txt", "a") as f:
          f.write("Starting of Synonyms of the Words from the Sentences: " + synonym_output + "\n")
          f.write("Starting of Antonyms of the Words from the Sentences: " + antonyms_output + "\n")
          f.close()
     '''

In [ ]:
nltk.download('punkt')  # 말뭉치 다운로드 필요
text_parser_synonym_antonym_finder(text="init failed run of stage")




아래는 pretrained Word2Vec 시험 용

In [ ]:
# PC 에서 GoogleNews-vectors-negative300.bin.gz 를 upload

from google.colab import files     # Colab에서 local PC내의 file 읽어 들이는 라이브러리
uploaded = files.upload()          # 파일을  선택

In [ ]:
# mount Drive to access data files
from google.colab import drive
drive.mount('./mount')    # 실행하면 'Colab의 Jupyter에서 Google Drive 파일에 액세스하도록 허용하시겠습니까?' 라는 질문이 나온다. 동의하면 됨
# 위 명령에 동의하고 완료되면 Mounted at ./mount 가 출력되고, 좌측에 mount  폴더가 생성된다.

In [ ]:
import gensim

# 우리가 업데이트하려고 하는 data가 다음 sentences에 있다고 합시다
# 그리고, 다음 데이터들은 모두 lower, strip, 공백으로 split되어 있다고 하죠.
sentences = ["I am a boy", "You are a girl"]
sentences = [s.lower().strip().split(" ") for s in sentences]
#---------------------------------------------
# 1) LOAD pre-trained key vector
# model을 load한 것이 아니기 때문에, 바로 training하는 것이 불가능하죠.
googleNews_filepath = "GoogleNews-vectors-negative300-SLIM.bin"
PreTrainedKeyvector = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format(
    googleNews_filepath, binary=True, limit=20
)
#---------------------------------------------
# 2) MAKE new word2vec model
# PreTrainedKeyvector와 `vector_size`가 같은 word2vec model을 생성해줌.
TranferedModel = gensim.models.Word2Vec(
    size=PreTrainedKeyvector.vector_size, min_count=1)
#---------------------------------------------
# 3) BUILD vocab by PreTrainedKeyvector word Vocabulary
# TranferedModel.build_vocab([[]]) # list of list
# ** list of list 로 넘겨줘야 함.
TranferedModel.build_vocab([PreTrainedKeyvector.wv.vocab.keys()])
#---------------------------------------------
# 4) UPDATE vocab by sentences
# update parameter를 True로 설정.
TranferedModel.build_vocab(sentences, update=True)
#---------------------------------------------
# 5) INITIALIZED word vector
# vocab에 있는 단어들의 vector를 `googleNews_filepath`에 있는 값으로 모두 업데이트해줌.
# lockf=1.0 : 보통은 vector를 update하지 못하도록 lock이 걸려 있는데, 여기서는 이걸 품.
TranferedModel.intersect_word2vec_format(
    googleNews_filepath, binary=True, lockf=1.0
)
#---------------------------------------------
# 6) TRAIN new data set.
print("== Before New data training")
print(TranferedModel.wv['boy'][:10])
TranferedModel.train(sentences, total_examples=len(sentences), epochs=100)
print("== After New data training")
print(TranferedModel.wv['boy'][:10])

In [ ]:
# 기존 word2vec 사용 시
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

model.most_similar(positive=['good', 'sad'], negative=['bad'])